<a href="https://colab.research.google.com/github/rohitpan/datasciencecoursera/blob/master/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.9 MB/s eta 0:00:00


In [2]:
!aws s3 cp --no-sign-request s3://amazon-pqa/amazon_pqa_headsets.json .

download: s3://amazon-pqa/amazon_pqa_headsets.json to ./amazon_pqa_headsets.json


In [3]:
!pip install --upgrade -q torch==2.0.0
!pip install --quiet transformers
!pip install --quiet sentence-transformers==2.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install  --quiet pandas scikit-learn flask-ngrok opensearch-py tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 18.2 MB/s eta 0:00:00


In [5]:
import json
import pandas as pd

def load_pqa(file_name, number_rows=10000):
    qa_list = []
    df = pd.DataFrame(columns=('question', 'answer', 'label'))
    with open(file_name) as f:
        i = 0
        previous_row_data = None
        for line in f:
            data = json.loads(line)
            df.loc[i] = [data['question_text'], data['answers'][0]['answer_text'], 1.0]
            i += 1
            if previous_row_data is not None:
                df.loc[i] = [data['question_text'], previous_row_data['answers'][0]['answer_text'], 0.0]
            previous_row_data = data
            i += 1
            if(i == number_rows*2):
                break
    return df

# Replace 'your-file.json' with the path to your dataset
qa_list = load_pqa('amazon_pqa_headsets.json')


In [6]:
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, InputExample

train_set, test_set = train_test_split(qa_list, test_size=0.2, shuffle=True)
training_set, validation_set = train_test_split(train_set, test_size=0.2)

def create_input_sample(data_set):
    train_samples = []
    for index, row in data_set.iterrows():
        input_example = InputExample(texts=[row['question'], row['answer']], label=row['label'])
        train_samples.append(input_example)
    return train_samples

training_samples = create_input_sample(training_set)
validation_samples = create_input_sample(validation_set)
test_samples = create_input_sample(test_set)


In [10]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import math
model_name = "sentence-transformers/distilbert-base-nli-stsb-mean-tokens"
train_batch_size = 16
num_epochs = 1
model_save_path = 'output/fine_tuned_'+model_name.replace("/", "-")

word_embedding_model = models.Transformer(model_name)

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

train_dataloader = DataLoader(training_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CoSEN

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(validation_samples, name='pqa-valucation')


warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) #10% of train data for warm-up



# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/800 [00:00<?, ?it/s]

In [11]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load test data
test_data = qa_list  # Replace with your dataset file
questions = test_data['question'].tolist()
answers = test_data['answer'].tolist()

# Load models
base_model = SentenceTransformer('sentence-transformers/distilbert-base-nli-stsb-mean-tokens')
fine_tuned_model = SentenceTransformer('output/fine_tuned_sentence-transformers-distilbert-base-nli-stsb-mean-tokens')

# Generate embeddings for questions and answers
def get_embeddings(model, texts):
    return model.encode(texts, convert_to_tensor=True)

base_question_embeddings = get_embeddings(base_model, questions)
base_answer_embeddings = get_embeddings(base_model, answers)

fine_tuned_question_embeddings = get_embeddings(fine_tuned_model, questions)
fine_tuned_answer_embeddings = get_embeddings(fine_tuned_model, answers)

def compute_similarity(question_embeddings, answer_embeddings):
    similarities = util.pytorch_cos_sim(question_embeddings, answer_embeddings)
    return similarities.diagonal().cpu().numpy()  # Diagonal gives similarity scores

base_similarities = compute_similarity(base_question_embeddings, base_answer_embeddings)
fine_tuned_similarities = compute_similarity(fine_tuned_question_embeddings, fine_tuned_answer_embeddings)

import numpy as np

# Compute metrics such as mean similarity
mean_base_similarity = np.mean(base_similarities)
mean_fine_tuned_similarity = np.mean(fine_tuned_similarities)

print(f"Mean Similarity (Base Model): {mean_base_similarity:.4f}")
print(f"Mean Similarity (Fine-Tuned Model): {mean_fine_tuned_similarity:.4f}")

Mean Similarity (Base Model): 0.2750
Mean Similarity (Fine-Tuned Model): 0.7380


Cosine Similarity
Mean Similarity (Base Model): 0.2750
Mean Similarity (Fine-Tuned Model): 0.467

Contrastive Similarity
Mean Similarity (Base Model): 0.2750
Mean Similarity (Fine-Tuned Model): 0.7380